In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df_hits = pd.read_csv("additional/hits_dataset.csv", delimiter="\t")
df_hits["hit"] = 1
df_hits

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next",['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],87,True,Solo,11,1,79,...,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,1
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,['26VFTg2z8YR0cCuwLzESi2'],['Halsey'],87,True,Solo,1,1,79,...,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,1
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,['0Y5tJX1MQlPlqiwlOH1tJY'],['Travis Scott'],86,True,Solo,3,1,79,...,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,1
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,"['246dkjvS1zLTtiykXe5h60', '1zNqQNIdeOUZHb8zbZ...","['Post Malone', 'Swae Lee']",92,False,Collaboration,2,2,79,...,0.55600,0.760,0.479,0.000000,0.0703,-5.574,0.0466,0.913,89.911,1
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,['20JZFwl6HVl6yg8a4H3ZqK'],['Panic! At The Disco'],86,False,Solo,4,1,78,...,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11954,5mz9pQZZXNpAw9CdQ7Bk8q,Don't Pity Me - Mono; 2003 Remaster,['6lHC2EQMEMZiEmSfFloarn'],['Peter And Gordon'],8,False,Solo,1,1,78,...,0.61900,0.500,0.654,0.000000,0.3320,-8.633,0.0394,0.403,107.326,1
11955,2H9CKpZiLDF223BbwehpDF,Hush Hush; Hush Hush - Main,['6wPhSqRtPu1UhRCDX5yaDJ'],['The Pussycat Dolls'],57,False,Solo,1,1,59,...,0.10500,0.609,0.790,0.000000,0.3140,-3.787,0.0663,0.664,129.334,1
11956,7zqM7Wm2xJLZ50qAnTLoHD,"Lions, Tigers & Bears",['7gSjFKpVmDgC2MMsnN8CYq'],['Jazmine Sullivan'],49,False,Solo,4,1,71,...,0.07880,0.576,0.515,0.000000,0.1180,-6.178,0.0295,0.407,95.406,1
11957,4DPlKVZ6R1EHLfnIsy9ouh,The Weight,"['0rXI0q8Cahq6numvPlloaq', '3RwQ26hR2tJtA8F9p2...","['Diana Ross & The Supremes', 'The Temptations']",2,False,Collaboration,23,2,79,...,0.37000,0.566,0.607,0.000000,0.6710,-7.532,0.0357,0.765,91.378,1


In [2]:
df_nonhits = pd.read_csv("additional/nonhits_dataset.csv", delimiter="\t")
df_nonhits["hit"] = 0
df_nonhits

C:\Users\lasse\AppData\Local\Temp\ipykernel_13916\2533836516.py:1: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nonhits = pd.read_csv("additional/nonhits_dataset.csv", delimiter="\t")


,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,1TEL6MlSSVLSdhOSddidlJ,needy,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],76,True,Solo,2,1,79,...,0.78,0.647,0.309,7.41e-06,0.202,-7.948,0.0366,0.195,87.045,0
1,4uTvPEr01pjTbZgl7jcKBD,NASA,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],76,False,Solo,3,1,79,...,0.451,0.747,0.458,0,0.252,-6.891,0.303,0.47,75.029,0
2,2hloaUoRonYssMuqLCBLTX,bloodline,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],74,True,Solo,4,1,79,...,0.0815,0.758,0.665,0.000157,0.216,-5.188,0.0774,0.643,151.976,0
3,3wFLWP0FcIqHK1wb1CPthQ,fake smile,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],73,True,Solo,5,1,79,...,0.329,0.45,0.619,0,0.319,-7.412,0.372,0.427,83.3,0
4,5Il6Oe7lr5XM7A0cWbVQtr,bad idea,['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],74,False,Solo,6,1,79,...,0.0268,0.847,0.579,0.0545,0.102,-5.314,0.0719,0.569,137.998,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899063,2tP0ryZParpdjvxubgXhTN,Love Is Here to Stay,"['3Dzj993UEz8Z5ovxuirzFO', '6RCsx4p5smZHYz2P5H...","['Lindsey Buckingham', 'Christine McVie']",16,False,Collaboration,5,2,1,...,0.366,0.446,0.658,2.49e-05,0.751,-8.362,0.0297,0.554,84.74,0
899064,7Ltni80U0w7bmYjXPNzSEa,Heinlein's Planet,['4urvwbid7NFoVes5PTx9lV'],['Tony Carey'],0,False,Solo,5,1,79,...,0.481,0.305,0.456,0.933,0.111,-20.199,0.0634,0.171,96.996,0
899065,1gQClXN1Dq4rj0wBzddcNA,Beds are Burning - Karaoke Lead Vocal Version,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,2,1,79,...,0.0118,0.852,0.648,0,0.0503,-7.189,0.048,0.559,120.15,0
899066,2jhxQrFNdVPkWL28X7ZpeK,Un homme heureux - Version karaoké instrumentale,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,1,1,79,...,0.973,0.664,0.353,0.787,0.0878,-12.303,0.0693,0.284,116.45,0


In [3]:
combined_df = pd.concat([df_hits, df_nonhits], axis=0)
combined_df.replace("-", float("nan"), inplace=True)
# Drop all rows with NaN values
combined_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next",['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],87,True,Solo,11,1,79,...,0.229,0.717,0.653,0.0,0.101,-5.634,0.0658,0.412,106.966,1
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,['26VFTg2z8YR0cCuwLzESi2'],['Halsey'],87,True,Solo,1,1,79,...,0.297,0.752,0.488,0.000009,0.0936,-7.05,0.0705,0.533,136.041,1
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,['0Y5tJX1MQlPlqiwlOH1tJY'],['Travis Scott'],86,True,Solo,3,1,79,...,0.00513,0.834,0.73,0.0,0.124,-3.714,0.222,0.446,155.008,1
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,"['246dkjvS1zLTtiykXe5h60', '1zNqQNIdeOUZHb8zbZ...","['Post Malone', 'Swae Lee']",92,False,Collaboration,2,2,79,...,0.556,0.76,0.479,0.0,0.0703,-5.574,0.0466,0.913,89.911,1
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,['20JZFwl6HVl6yg8a4H3ZqK'],['Panic! At The Disco'],86,False,Solo,4,1,78,...,0.193,0.579,0.904,0.0,0.064,-2.729,0.0618,0.681,82.014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899063,2tP0ryZParpdjvxubgXhTN,Love Is Here to Stay,"['3Dzj993UEz8Z5ovxuirzFO', '6RCsx4p5smZHYz2P5H...","['Lindsey Buckingham', 'Christine McVie']",16,False,Collaboration,5,2,1,...,0.366,0.446,0.658,2.49e-05,0.751,-8.362,0.0297,0.554,84.74,0
899064,7Ltni80U0w7bmYjXPNzSEa,Heinlein's Planet,['4urvwbid7NFoVes5PTx9lV'],['Tony Carey'],0,False,Solo,5,1,79,...,0.481,0.305,0.456,0.933,0.111,-20.199,0.0634,0.171,96.996,0
899065,1gQClXN1Dq4rj0wBzddcNA,Beds are Burning - Karaoke Lead Vocal Version,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,2,1,79,...,0.0118,0.852,0.648,0,0.0503,-7.189,0.048,0.559,120.15,0
899066,2jhxQrFNdVPkWL28X7ZpeK,Un homme heureux - Version karaoké instrumentale,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,1,1,79,...,0.973,0.664,0.353,0.787,0.0878,-12.303,0.0693,0.284,116.45,0


In [4]:
# Check for NaN values in all columns
nan_columns = combined_df.columns[combined_df.isna().any()].tolist()

# Print columns with NaN values
print("Columns with NaN values:")
print(nan_columns)


Columns with NaN values:
['duration_ms', 'key', 'mode', 'time_signature', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo']


In [5]:
from sklearn.impute import SimpleImputer

combined_df.replace("-", float("nan"), inplace=True)   ## evtl hier schon mean replacement???
# combined_df.dropna(inplace=True)   # drop nan value rows
#Fill missing values with mean for each numeric attribute
imputer = SimpleImputer(strategy='mean')
for col in nan_columns:
    combined_df[col] = imputer.fit_transform(combined_df[[col]])

combined_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next",['66CXWjxzNUsdJxJ2JdwvnR'],['Ariana Grande'],87,True,Solo,11,1,79,...,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,1
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,['26VFTg2z8YR0cCuwLzESi2'],['Halsey'],87,True,Solo,1,1,79,...,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,1
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,['0Y5tJX1MQlPlqiwlOH1tJY'],['Travis Scott'],86,True,Solo,3,1,79,...,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,1
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,"['246dkjvS1zLTtiykXe5h60', '1zNqQNIdeOUZHb8zbZ...","['Post Malone', 'Swae Lee']",92,False,Collaboration,2,2,79,...,0.55600,0.760,0.479,0.000000,0.0703,-5.574,0.0466,0.913,89.911,1
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,['20JZFwl6HVl6yg8a4H3ZqK'],['Panic! At The Disco'],86,False,Solo,4,1,78,...,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899063,2tP0ryZParpdjvxubgXhTN,Love Is Here to Stay,"['3Dzj993UEz8Z5ovxuirzFO', '6RCsx4p5smZHYz2P5H...","['Lindsey Buckingham', 'Christine McVie']",16,False,Collaboration,5,2,1,...,0.36600,0.446,0.658,0.000025,0.7510,-8.362,0.0297,0.554,84.740,0
899064,7Ltni80U0w7bmYjXPNzSEa,Heinlein's Planet,['4urvwbid7NFoVes5PTx9lV'],['Tony Carey'],0,False,Solo,5,1,79,...,0.48100,0.305,0.456,0.933000,0.1110,-20.199,0.0634,0.171,96.996,0
899065,1gQClXN1Dq4rj0wBzddcNA,Beds are Burning - Karaoke Lead Vocal Version,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,2,1,79,...,0.01180,0.852,0.648,0.000000,0.0503,-7.189,0.0480,0.559,120.150,0
899066,2jhxQrFNdVPkWL28X7ZpeK,Un homme heureux - Version karaoké instrumentale,['7yv4DJOCmilSbxxsdQgXWk'],['Pictomusic'],0,False,Solo,1,1,79,...,0.97300,0.664,0.353,0.787000,0.0878,-12.303,0.0693,0.284,116.450,0


In [6]:
# Check for NaN values in all columns
nan_columns = combined_df.columns[combined_df.isna().any()].tolist()

# Print columns with NaN values
print("Columns with NaN values:")
print(nan_columns)


Columns with NaN values:
[]


In [7]:
from sklearn.model_selection import train_test_split
combined_df['date'] = pd.to_datetime(combined_df['release_date'])
combined_df.sort_values(by="date", inplace=True)
#border_day = combined_df["date"].iloc[-1]  - pd.DateOffset(years=4)
#combined_df = combined_df[(combined_df["date"] >= border_day)]

split_day = combined_df["date"].iloc[-1]  - pd.DateOffset(years=1)
train_df = combined_df[(combined_df["date"]  < split_day)]
test_df = combined_df[(combined_df["date"]  >= split_day)]
#train_df, test_df = train_test_split(combined_df, test_size=1/6, shuffle=False)

In [8]:
combined_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit,date
804676,7kXUEJmfvRXbzxOC0pHQgb,I Can't Believe I'm Losing You,['6eVCRmRG2HcfL4yx77A9WK'],['Jack Jones'],5,False,Solo,12,1,79,...,0.180,0.369,0.000000,0.1500,-9.797,0.0299,0.224,81.701,0,1995-01-01
465447,1xdRFXv3ytSOdj8GuT3Rge,Thank You Lord (Interlude),['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],20,False,Solo,10,1,77,...,0.370,0.284,0.000000,0.5580,-14.443,0.0378,0.649,130.985,0,1995-01-01
465446,5wlQIBbKIWoB7AnSw09C6C,All This Love,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],26,False,Solo,9,1,77,...,0.605,0.604,0.000153,0.0693,-9.214,0.0480,0.503,145.483,0,1995-01-01
377956,40uHAEjua1YmyKIJ6k0iVa,Bo Bo Wa,['3z6Uf5IZkN4TogRpRpgD84'],['Gary Wright'],0,False,Solo,8,1,79,...,0.481,0.652,0.008670,0.0397,-10.981,0.0364,0.795,195.791,0,1995-01-01
465445,2zmMWWgwCXYarkO2SbyPZP,Come Over,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],38,False,Solo,7,1,77,...,0.534,0.563,0.000001,0.5740,-8.975,0.0601,0.414,124.149,0,1995-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11098,0xAqhyWvTkfcdFgm7dozMO,(I Wanna) Testify (Mono Single Version),['1Ve5PoI7i5jIIP1xudo6AB'],['The Parliaments'],7,False,Solo,10,1,79,...,0.668,0.702,0.000000,0.0521,-5.750,0.0330,0.754,105.227,1,2019-08-23
10903,6zhg8TnVKi90ITDzFKwPr0,Swingtown - Alternate Version,['6QtGlUje9TIkLrgPZrESuk'],['Steve Miller Band'],26,False,Solo,5,1,79,...,0.459,0.764,0.000544,0.2440,-6.662,0.0354,0.439,124.557,1,2019-08-23
5856,2GvkFSjRXV0Ii3uYnHiQ74,Take My Advice,['40PLSl99xXqAQsCQQPmfyM'],['Kym Sims'],0,False,Solo,10,1,73,...,0.737,0.814,0.392000,0.0844,-10.396,0.0307,0.803,120.010,1,2019-08-23
5080,0lej8CemYKkWVHg77Xe1ky,A Man Holdin' on (To a Woman Lettin’ Go),"['1Fj0R2t4HaJa3oUe8azB8R', '5LDnSV99gbDSsMX3Dc...","['Ty Herndon', 'Anita Cochran']",14,False,Collaboration,7,2,79,...,0.611,0.429,0.000014,0.1120,-8.619,0.0298,0.237,128.007,1,2019-08-23


In [9]:
# Specify the columns
columns = ['artist_id', 'artist_name']

# Initialize an empty DataFrame
df_artists_train = pd.DataFrame(columns={col: [] for col in columns})

count = 0
artist_set = set()

for _, row in train_df.iterrows():
    current_artist_id_list = eval(row["id_artists"])
    current_artist_name_list = eval(row["name_artists"])
    for cur_art_id, cur_art_name in zip(current_artist_id_list, current_artist_name_list):
        if cur_art_id not in artist_set:
            artist_set.add(cur_art_id)
            df_artists_train.loc[len(df_artists_train)] = [cur_art_id, cur_art_name]
            count += 1

print("amount of artists:", count)
df_artists_train

amount of artists: 24067


,artist_id,artist_name
0,6eVCRmRG2HcfL4yx77A9WK,Jack Jones
1,5NDMothbpdpq2xHqSjrrWn,Faith Evans
2,3z6Uf5IZkN4TogRpRpgD84,Gary Wright
3,2UuKUaomk3bZ0nHM3Y3FdW,Bud Shank
4,6S0GHTqz5sxK5f9HtLXn9q,Garbage
...,...,...
24062,2D7RkvtKKb6E5UmbjQM1Jd,Georges Bizet
24063,5UvGRahK1OytDXGTzdf6hV,Humphrey Burton
24064,0MIlVcFgL7pDFqaZRmYDeE,Weiss
24065,2iG0HNCECG2DxXxSUcMhEX,Naxxos


In [10]:
import networkx as nx

all_artist_ids = artist_set.copy()

G = nx.Graph()

for id_art_cur in all_artist_ids:
    artist_name = df_artists_train[(df_artists_train["artist_id"] == id_art_cur)]["artist_name"]
    artist_info = df_artists_train[df_artists_train["artist_id"] == id_art_cur]
    if not artist_info.empty:  # Check if artist info exists
        # Filter out artists who are not successful
        node_attrs = {"name": artist_name.iloc[0]}
        G.add_node(id_art_cur, **node_attrs)
    else:
        print("Artist info not found for ID:", id_art_cur)

for _, row in train_df.iterrows():
    artist_ids = list(eval(row["id_artists"]))
    artist_names = list(eval(row["name_artists"]))
    # Filter out songs that do not have more than one artist in their execution
    if len(artist_ids) > 1:
        for i in range(len(artist_ids)):
            for j in range(i+1, len(artist_ids)):
                artist_id_1 = str(artist_ids[i])
                lable1 = str(artist_names[i])
                artist_id_2 = str(artist_ids[j])
                lable2 = str(artist_names[j])
                # Check if the edge already exists
                if G.has_edge(artist_id_1, artist_id_2):
                    # If the edge already exists, increment the weight by 1
                    G[artist_id_1][artist_id_2]['weight'] += 2
                else:
                    # Add a new edge with weight 1
                    G.add_edge(artist_id_1, artist_id_2, lable1=lable1, lable2=lable2, weight=2, song_id=str(row["song_id"]), song_name=str(row["song_name"]), explicit=bool(row["explicit"]), song_type=str(row["song_type"]), track_number=int(row["track_number"]), num_artists=int(row["num_artists"]), num_available_markets=int(row["num_available_markets"]), duration_ms=int(row["duration_ms"]), popularity=int(row["popularity"]), release_date=str(row["release_date"]), key = int(row["key"]), mode = int(row["mode"]), time_signature = int(row["time_signature"]), acousticness = float(row["acousticness"]), danceability = float(row["danceability"]), energy = float(row["energy"]), instrumentalness = float(row["instrumentalness"]), liveness = float(row["liveness"]),loudness = float(row["loudness"]), speechiness = float(row["speechiness"]), valence = float(row["valence"]), tempo = float(row["tempo"]))

In [11]:
# Assuming G is your NetworkX graph

# Extract nodes and their attributes from the graph
nodes_with_attributes = [(G.nodes[node], node) for node in G.nodes()]

# Create a DataFrame from the nodes and their attributes
df_nodes_train = pd.DataFrame(nodes_with_attributes, columns=['Attributes', "Spotify ID"])

# Normalize the 'Attributes' column to expand it into separate columns
df_attributes = pd.json_normalize(df_nodes_train['Attributes'])

# Combine the original 'Node' column with the expanded attributes
df_nodes_train = pd.concat([df_attributes, df_nodes_train['Spotify ID']], axis=1)

# Display the DataFrame
df_nodes_train

,name,Spotify ID
0,The Kenny Clarke-Francy Boland Big Band,7C6sfpvSQJ9V0i4hDW9ciO
1,Mark Josher,4u1PPGrzg0ZG9GO69BAR8Q
2,Tim Janssens,1ysOGN05BRRIId5fOZ92y1
3,El DeBarge,5xO4f5QyPMYnCPRyRveBSD
4,Jeanne & The Darlings,6qJgLKcdcL8zxzfsWRylRk
...,...,...
24062,Sångensemblen Amanda,6lLjsatj15QODusCJkLR7v
24063,Dirty Deeds,4dHsqFrT3KLR7LhyboGTDN
24064,Tisha Campbell Martin,6mx0D9tlGW7OCTlrgDKaxs
24065,Hannah Elless,7q4Ulw0Vfty0d8g8WoNL7A


In [12]:
df_nodes_train.to_csv("network_created/nodes_real_train.csv", sep='\t', index=False)

In [13]:
# Extract edges and their attributes from the graph
edges_with_attributes = [(u, v, G.edges[u, v]) for u, v in G.edges()]

# Create a DataFrame from the edges and their attributes
df_edges_train = pd.DataFrame(edges_with_attributes, columns=['Source', 'Target', 'Attributes'])

# Normalize the 'Attributes' column to expand it into separate columns
df_edge_attributes = pd.json_normalize(df_edges_train['Attributes'])

# Combine the original 'Attributes' column with the expanded attributes
df_edges_train = pd.concat([df_edges_train[['Source', 'Target']], df_edge_attributes], axis=1)
df_edges_train

,Source,Target,lable1,lable2,weight,song_id,song_name,explicit,song_type,track_number,...,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,7C6sfpvSQJ9V0i4hDW9ciO,0FMucZsEnCxs5pqBjHjIc8,The Kenny Clarke-Francy Boland Big Band,Stan Getz,2,7pKhgISg15Gu6fvtXQk9Xd,Symptones,False,Collaboration,9,...,4,0.70400,0.374,0.336,0.04280,0.176,-12.895,0.0315,0.3120,101.082
1,4u1PPGrzg0ZG9GO69BAR8Q,3DmG65yHQsMms7WAvrZOdt,Enigma,Mark Josher,2,2JemWSF5aQOlH52TK5MrOc,The Die Is Cast,False,Collaboration,4,...,4,0.01140,0.247,0.486,0.11700,0.252,-12.656,0.0412,0.0672,104.036
2,1ysOGN05BRRIId5fOZ92y1,21E3waRsmPlU7jZsS13rcj,Ne-Yo,Tim Janssens,4,0LQIiWL5eDKHn2OLNe3ivE,Because Of You - Sunfreakz Remix,False,Collaboration,4,...,4,0.00278,0.711,0.864,0.00811,0.670,-3.835,0.0451,0.8060,126.004
3,5xO4f5QyPMYnCPRyRveBSD,7AjokpCIqf0gHMZJlodw2k,Suga Free,El DeBarge,2,0oWJiWYnKrq4fHUZAuZmDq,On My Way,True,Collaboration,10,...,4,0.03970,0.533,0.706,0.00000,0.140,-7.368,0.3790,0.6690,86.158
4,5xO4f5QyPMYnCPRyRveBSD,3q7HBObVc0L8jNeTe5Gofh,El DeBarge,50 Cent,4,42FKMEfHg9NRNQpg9iFw1t,Format,False,Collaboration,4,...,4,0.05410,0.605,0.769,0.00000,0.356,-4.435,0.2610,0.3950,95.558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25100,4isewEHTP7Az2QheDdBQ6J,3wyVrVrFCkukjdVIdirGVY,24hrs,Lil Pump,2,1gUJxDCuVx3DFlADiZl4ry,Lie Detector (feat. Lil Pump),True,Collaboration,1,...,4,0.29800,0.841,0.829,0.00000,0.196,-3.290,0.0637,0.8190,77.518
25101,4yiQZ8tQPux8cPriYMWUFP,5a2EaR3hamoenG9rDuVn8j,Prince,Gwen Stefani,4,3CJEtUMAzh9B54rSvuiK5d,"So Far, So Pleased (feat. Gwen Stefani)",False,Collaboration,6,...,4,0.00810,0.668,0.827,0.01510,0.179,-4.380,0.0746,0.5420,123.001
25102,5a2EaR3hamoenG9rDuVn8j,6ueGR6SWhUJfvEhqkvMsVs,Janelle Monáe,Prince,2,1S2fiTIYw8cy2lH3JBXN64,Givin Em What They Love (feat. Prince),False,Collaboration,2,...,4,0.04140,0.640,0.600,0.01470,0.468,-7.719,0.0492,0.4830,144.026
25103,7wU2WGCJ8HxkekHHE2QLul,0TnOYISbd1XYRBk9myaseg,Pitbull,Fuego,2,406DbQLmjelaICKQhJ4BFM,Mami Mami,False,Collaboration,6,...,4,0.05560,0.722,0.756,0.00000,0.163,-5.592,0.0725,0.5460,89.969


In [14]:
# Save edge DataFrame to CSV
df_edges_train.to_csv('network_created/edges_real_train.csv', sep='\t', index=False)

In [15]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)


Number of nodes: 24067
Number of edges: 25105


In [16]:
# Specify the columns
columns = ['artist_id', 'artist_name']

# Initialize an empty DataFrame
df_artists_test = pd.DataFrame(columns={col: [] for col in columns})

count = 0
artist_set = set()

for _, row in test_df.iterrows():
    current_artist_id_list = eval(row["id_artists"])
    current_artist_name_list = eval(row["name_artists"])
    for cur_art_id, cur_art_name in zip(current_artist_id_list, current_artist_name_list):
        if cur_art_id not in artist_set:
            artist_set.add(cur_art_id)
            df_artists_test.loc[len(df_artists_test)] = [cur_art_id, cur_art_name]
            count += 1

print("amount of artists:", count)
df_artists_test

amount of artists: 5902


,artist_id,artist_name
0,0KMJUP87x4ERfwSNYvkttw,Sandgrounder
1,3OpJxAHfT0KsJzQqdX5SLb,Ginger Tunes
2,6BkcAbUkfIBM4XudxieMq8,Dr. Alban
3,2iAAXHZCgL2E1zqj204GEh,Jacci McGhee
4,21VBNkZvKxrI9zmfFRxXvn,8-Bit Arcade
...,...,...
5897,3Y9xnCbmXGhmpJymwpnxCz,Crispian St. Peters
5898,5NDMothbpdpq2xHqSjrrWn,Faith Evans
5899,1Ve5PoI7i5jIIP1xudo6AB,The Parliaments
5900,40PLSl99xXqAQsCQQPmfyM,Kym Sims


In [17]:
import networkx as nx

all_artist_ids = artist_set.copy()

G2 = nx.Graph()

for id_art_cur in all_artist_ids:
    artist_name = df_artists_test[(df_artists_test["artist_id"] == id_art_cur)]["artist_name"]
    artist_info = df_artists_test[df_artists_test["artist_id"] == id_art_cur]
    if not artist_info.empty:  # Check if artist info exists
        # Filter out artists who are not successful
        node_attrs = {"name": artist_name.iloc[0]}
        G2.add_node(id_art_cur, **node_attrs)
    else:
        print("Artist info not found for ID:", id_art_cur)

for _, row in test_df.iterrows():
    artist_ids = list(eval(row["id_artists"]))
    artist_names = list(eval(row["name_artists"]))
    # Filter out songs that do not have more than one artist in their execution
    if len(artist_ids) > 1:
        for i in range(len(artist_ids)):
            for j in range(i+1, len(artist_ids)):
                artist_id_1 = str(artist_ids[i])
                lable1 = str(artist_names[i])
                artist_id_2 = str(artist_ids[j])
                lable2 = str(artist_names[j])
                # Check if the edge already exists
                if G2.has_edge(artist_id_1, artist_id_2):
                    # If the edge already exists, increment the weight by 1
                    G2[artist_id_1][artist_id_2]['weight'] += 2
                else:
                    # Add a new edge with weight 1
                    G2.add_edge(artist_id_1, artist_id_2, lable1=lable1, lable2=lable2, weight=2, song_id=str(row["song_id"]), song_name=str(row["song_name"]), explicit=bool(row["explicit"]), song_type=str(row["song_type"]), track_number=int(row["track_number"]), num_artists=int(row["num_artists"]), num_available_markets=int(row["num_available_markets"]), duration_ms=int(row["duration_ms"]), popularity=int(row["popularity"]), release_date=str(row["release_date"]), key = int(row["key"]), mode = int(row["mode"]), time_signature = int(row["time_signature"]), acousticness = float(row["acousticness"]), danceability = float(row["danceability"]), energy = float(row["energy"]), instrumentalness = float(row["instrumentalness"]), liveness = float(row["liveness"]),loudness = float(row["loudness"]), speechiness = float(row["speechiness"]), valence = float(row["valence"]), tempo = float(row["tempo"]))

In [18]:
# Assuming G is your NetworkX graph

# Extract nodes and their attributes from the graph
nodes_with_attributes = [(G2.nodes[node], node) for node in G2.nodes()]

# Create a DataFrame from the nodes and their attributes
df_nodes_test = pd.DataFrame(nodes_with_attributes, columns=['Attributes', "Spotify ID"])

# Normalize the 'Attributes' column to expand it into separate columns
df_attributes = pd.json_normalize(df_nodes_test['Attributes'])

# Combine the original 'Node' column with the expanded attributes
df_nodes_test = pd.concat([df_attributes, df_nodes_test['Spotify ID']], axis=1)

# Display the DataFrame
df_nodes_test

,name,Spotify ID
0,Jacob Banks,0AepkoQhYvkjEzzwIcGxdV
1,Lola Khan,6AgXlZ1FlOLAqeR8qfDfv9
2,E.J. Carter,7m3eGk2DFIvZrUbgjaoWxC
3,Sonny Landreth,7aHLYoLUMdcl0HiT5k0Brz
4,George McCrae,6oV3BNm1Gj2GGgpYknc5TN
...,...,...
5897,aboutagirl,1Rm8cDAHAL2mo9H7sUxohj
5898,Tom Cochrane,5Jj4mqGYiplyowPLKkJLHt
5899,Lil Advil,2AHPBn0FrIthLv3ijcnQkt
5900,The Honey Sweets,4ZhGdJTksEoPOvjvkPUoj0


In [19]:
df_nodes_test.to_csv("network_created/nodes_real_test.csv", sep='\t', index=False)

In [20]:
# Extract edges and their attributes from the graph
edges_with_attributes = [(u, v, G2.edges[u, v]) for u, v in G2.edges()]

# Create a DataFrame from the edges and their attributes
df_edges_test = pd.DataFrame(edges_with_attributes, columns=['Source', 'Target', 'Attributes'])

# Normalize the 'Attributes' column to expand it into separate columns
df_edge_attributes = pd.json_normalize(df_edges_test['Attributes'])

# Combine the original 'Attributes' column with the expanded attributes
df_edges_test = pd.concat([df_edges_test[['Source', 'Target']], df_edge_attributes], axis=1)
df_edges_test

,Source,Target,lable1,lable2,weight,song_id,song_name,explicit,song_type,track_number,...,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,0AepkoQhYvkjEzzwIcGxdV,24eDfi2MSYo3A87hCcgpIL,Jacob Banks,Ludwig Goransson,2,0QTOw32h1G2LD8EFExxedV,You Might Find Me,False,Collaboration,11,...,5,0.94000,0.393,0.0118,0.012100,0.1060,-22.988,0.0500,0.0596,108.236
1,0AepkoQhYvkjEzzwIcGxdV,3NPpFNZtSTHheNBaWC82rB,X Ambassadors,Jacob Banks,2,57cAALfX35bWaURrtJnumR,Baptize Me,False,Collaboration,5,...,4,0.03230,0.562,0.5110,0.000244,0.2060,-7.960,0.0375,0.1340,135.009
2,7m3eGk2DFIvZrUbgjaoWxC,4Q5sPmM8j4SpMqL4UA1DtS,Lil' Flip,E.J. Carter,34,650CojXnzXGLeaMRZJOQ6x,Propane - Remix,True,Collaboration,2,...,4,0.55800,0.926,0.7960,0.000000,0.0742,-5.095,0.2460,0.7500,130.013
3,7aHLYoLUMdcl0HiT5k0Brz,0JvNeHGFrPqynXe5EszItO,Peter Frampton Band,Sonny Landreth,2,04YHvDofQY0u2FjqrRCeYn,The Thrill Is Gone,False,Collaboration,7,...,4,0.50100,0.396,0.6230,0.000479,0.3020,-7.390,0.0651,0.5250,177.395
4,1v3tdpIdBSW14rHUfiEVOv,4BYxqVkZyFjtik7crYLg5Q,Chris Young,Lauren Alaina,2,5DrmJ25aHuPBImk1kLp5Of,Town Ain't Big Enough,False,Collaboration,1,...,4,0.06380,0.441,0.8260,0.001620,0.0944,-4.175,0.0400,0.5770,75.982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,5C4PDR4LnhZTbVnKWXuDKD,1d6dwipPrsFSJVmFTTdFSS,Paulina Rubio,Morat,4,3p770TQzJS3rNUHlDGV54Q,Mi Nuevo Vicio,False,Collaboration,5,...,4,0.03510,0.652,0.8950,0.000012,0.0570,-6.163,0.0307,0.8810,144.021
3648,6i3IIZPo1XeS7X6pTHzSN0,0kEcxP5ezzvP5RcO51TCBD,Nitty,Layla DaVias,4,3PmOsURgHBUntgFx2OUU8m,We Are,False,Collaboration,1,...,4,0.00392,0.695,0.8000,0.000004,0.5380,-7.730,0.0716,0.4820,92.028
3649,4O15NlyKLIASxsJ0PrXPfz,3wyVrVrFCkukjdVIdirGVY,Lil Pump,Lil Uzi Vert,6,7q8cy4uoA2itYATsKeXhcq,Multi Millionaire (feat. Lil Uzi Vert),True,Collaboration,1,...,4,0.02260,0.856,0.8200,0.000000,0.1680,-3.813,0.0962,0.7230,146.009
3650,21451j1KhjAiaYKflxBjr1,1r4hJ1h58CWwUQe3MxPuau,Maluma,Zion & Lennox,2,40kh4oBp5jEgdMN4WhvtCN,Extrañándote (feat. Zion & Lennox),False,Collaboration,7,...,4,0.04150,0.831,0.7620,0.000000,0.1290,-3.336,0.0792,0.5330,91.998


In [21]:
# Save edge DataFrame to CSV
df_edges_test.to_csv('network_created/edges_real_test.csv', sep='\t', index=False)

In [22]:
num_nodes = G2.number_of_nodes()
num_edges = G2.number_of_edges()
print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)


Number of nodes: 5902
Number of edges: 3652


In [23]:
train_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit,date
804676,7kXUEJmfvRXbzxOC0pHQgb,I Can't Believe I'm Losing You,['6eVCRmRG2HcfL4yx77A9WK'],['Jack Jones'],5,False,Solo,12,1,79,...,0.180,0.369,0.000000,0.1500,-9.797,0.0299,0.2240,81.701,0,1995-01-01
465447,1xdRFXv3ytSOdj8GuT3Rge,Thank You Lord (Interlude),['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],20,False,Solo,10,1,77,...,0.370,0.284,0.000000,0.5580,-14.443,0.0378,0.6490,130.985,0,1995-01-01
465446,5wlQIBbKIWoB7AnSw09C6C,All This Love,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],26,False,Solo,9,1,77,...,0.605,0.604,0.000153,0.0693,-9.214,0.0480,0.5030,145.483,0,1995-01-01
377956,40uHAEjua1YmyKIJ6k0iVa,Bo Bo Wa,['3z6Uf5IZkN4TogRpRpgD84'],['Gary Wright'],0,False,Solo,8,1,79,...,0.481,0.652,0.008670,0.0397,-10.981,0.0364,0.7950,195.791,0,1995-01-01
465445,2zmMWWgwCXYarkO2SbyPZP,Come Over,['5NDMothbpdpq2xHqSjrrWn'],['Faith Evans'],38,False,Solo,7,1,77,...,0.534,0.563,0.000001,0.5740,-8.975,0.0601,0.4140,124.149,0,1995-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111731,1TDIEgiZDIedT13NQ2mAXg,Northern Soul,['7JfutRemdlJGyQsIZ7wWQ9'],['Workout Buddy'],0,False,Solo,20,1,79,...,0.801,0.452,0.895000,0.0366,-11.564,0.0725,0.3530,121.983,0,2018-08-25
111730,4EJjPeKdHIZBEd3LJFEG3J,Chilled Life,['7JfutRemdlJGyQsIZ7wWQ9'],['Workout Buddy'],0,False,Solo,19,1,79,...,0.792,0.870,0.664000,0.0627,-9.842,0.0448,0.4110,124.016,0,2018-08-25
111729,1dGHkh8SvzrJJB5i0lMf37,The Messiahs Birthday,['7JfutRemdlJGyQsIZ7wWQ9'],['Workout Buddy'],0,False,Solo,18,1,79,...,0.761,0.588,0.913000,0.1130,-13.552,0.0690,0.1090,123.982,0,2018-08-25
111728,1llc2q5P1fAeBvEf3xWK0s,Flamma,['7JfutRemdlJGyQsIZ7wWQ9'],['Workout Buddy'],0,False,Solo,17,1,79,...,0.637,0.969,0.836000,0.3790,-4.583,0.0436,0.0719,128.098,0,2018-08-25


In [24]:
test_df

,song_id,song_name,id_artists,name_artists,popularity,explicit,song_type,track_number,num_artists,num_available_markets,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,hit,date
563810,6R3RXAvtypsoN6jWsjeabw,Nordic Shores,['0KMJUP87x4ERfwSNYvkttw'],['Sandgrounder'],0,False,Solo,5,1,79,...,0.646,0.490,0.895000,0.2230,-16.153,0.0525,0.8420,95.016,0,2018-08-26
563811,2Tb4RA2rzI3Xp97Q3EuhZ6,Wave,['3OpJxAHfT0KsJzQqdX5SLb'],['Ginger Tunes'],0,False,Solo,6,1,79,...,0.668,0.414,0.902000,0.1120,-10.019,0.0361,0.0399,121.084,0,2018-08-26
563812,2ADv9UhiY8pACXvwxP9mr0,Waltz for Joshua,['3OpJxAHfT0KsJzQqdX5SLb'],['Ginger Tunes'],0,False,Solo,7,1,79,...,0.650,0.235,0.887000,0.1490,-12.020,0.0346,0.0391,116.265,0,2018-08-26
641117,2RcAviQBeJrtZDZ7P1Co9k,Elmoped,['6BkcAbUkfIBM4XudxieMq8'],['Dr. Alban'],28,False,Solo,1,1,79,...,0.851,0.568,0.000000,0.1450,-7.159,0.0589,0.4710,119.981,0,2018-08-26
563813,7DnbauUP88t63cDXUrdovz,Black Orpheus,['3OpJxAHfT0KsJzQqdX5SLb'],['Ginger Tunes'],0,False,Solo,8,1,79,...,0.752,0.435,0.901000,0.0886,-10.619,0.0347,0.1250,122.128,0,2018-08-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11098,0xAqhyWvTkfcdFgm7dozMO,(I Wanna) Testify (Mono Single Version),['1Ve5PoI7i5jIIP1xudo6AB'],['The Parliaments'],7,False,Solo,10,1,79,...,0.668,0.702,0.000000,0.0521,-5.750,0.0330,0.7540,105.227,1,2019-08-23
10903,6zhg8TnVKi90ITDzFKwPr0,Swingtown - Alternate Version,['6QtGlUje9TIkLrgPZrESuk'],['Steve Miller Band'],26,False,Solo,5,1,79,...,0.459,0.764,0.000544,0.2440,-6.662,0.0354,0.4390,124.557,1,2019-08-23
5856,2GvkFSjRXV0Ii3uYnHiQ74,Take My Advice,['40PLSl99xXqAQsCQQPmfyM'],['Kym Sims'],0,False,Solo,10,1,73,...,0.737,0.814,0.392000,0.0844,-10.396,0.0307,0.8030,120.010,1,2019-08-23
5080,0lej8CemYKkWVHg77Xe1ky,A Man Holdin' on (To a Woman Lettin’ Go),"['1Fj0R2t4HaJa3oUe8azB8R', '5LDnSV99gbDSsMX3Dc...","['Ty Herndon', 'Anita Cochran']",14,False,Collaboration,7,2,79,...,0.611,0.429,0.000014,0.1120,-8.619,0.0298,0.2370,128.007,1,2019-08-23
